## 검증 세트
다른 하이퍼파라미터로 여러 모델을 만들어서 테스트셋으로 평가하게 되면 결국 테스트셋에 모델을 맞추는 결과가 될 수 있다. 이런 문제를 해결하기 위해 훈련셋에서 **검증 세트(validation set)**를 따로 나누어 검증셋으로 계속해서 훈련한 뒤 마지막에 테스트셋으로 평가를 하는 방법이 있다

In [16]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine[['class']].to_numpy()

# 훈련셋과 테스트셋 분리
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

# 훈련셋 안에서 검증셋 나누기
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)
print(sub_input.shape, val_input.shape)
print()

# 모델 훈련 및 평가
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

(4157, 3) (1040, 3)

0.9971133028626413
0.864423076923077


## 교차 검증
**교차 검증(cross validation)**은 검증셋을 떼내 평가하는 과정을 여러 번 반복하고 최종 검증 점수를 얻는다. 훈련셋을 5등분 한다고 했을 때,

□□□□□ 여기에서 칠한 부분을 검증셋이라고 하면 5-폴드 교차 검증은 다음과 같이 진행된다

■□□□□

□■□□□

□□■□□

□□□■□

□□□□■

이렇게 5번의 모델 평가를 거치고 5개의 평가 점수의 평균으로 검증 점수를 매긴다. 보통 5-폴드나 10-폴드를 많이 사용한다. 사이킷런에는 cross_validate() 메서드로 교차 검증 알고리즘을 제공한다

In [17]:
from sklearn.model_selection import cross_validate

# fit_time: 훈련 시간, score_time: 검증 시간, test_score: 각 검증 시 검증 점수
scores = cross_validate(dt, train_input, train_target)
for i in scores:
    print(i, scores[i])
print()

import numpy as np

print(np.mean(scores['test_score']))

fit_time [0.00925612 0.00697565 0.00735974 0.00717926 0.00686502]
score_time [0.00078726 0.00065994 0.00086308 0.0006628  0.00066924]
test_score [0.86923077 0.84615385 0.87680462 0.84889317 0.83541867]

0.855300214703487


교차 검증시 주의 점은 cross_validate()는 훈련셋을 섞어 폴드를 나누지 않는다. train_test_split() 메서드는 전체 데이터를 섞어주기 때문에 따로 섞을 필요가 없지만, 교차 검증 시에는 분할기(splitter)를 지정하여 훈련셋을 섞을 수 있다

cross_validate() 메서드는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다.

In [18]:
# 위와 같은 교차검증
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))
print()

# 훈련셋 섞은 후 10-폴드 교차검증
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.855300214703487

0.8574181117533719


## 하이퍼파라미터 튜닝
모델이 학습할 수 없어 사람이 지정해줘야 하는 파라미터를 하이퍼파라미터라 한다. 이 파라미터를 튜닝하는 방법은 먼저 라이브러리가 제공하는 기본값을 그대로 사용한 후, 검증셋의 점수나 교차 검증을 통해 매개변수를 조금씩 바꿔보면서 훈련을 진행하는 것이다. 이때 주의할 점은 하나의 최적의 하이퍼파라미터를 찾았다고 그 변수를 고정해놓고 다음 하이퍼파라미터를 찾는 것은 안된다. 모든 하이퍼파라미터를 동시에 바꿔가면서 최적의 값을 찾아야 한다. 이렇게 여러 하이퍼파라미터의 최적값을 찾기위해 **그리드 서치(Grid Search)**를 사용할 수 있다

사이킷런의 GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행하므로 별도로 cross_validate() 메서드를 호출할 필요가 없다. 여기서는 간단하게 min_impurity_decrease 매개변수의 최적값을 찾아보도록 하자

In [22]:
# GridSearchCV 임포트 후 탐색할 매개변수와 탐색할 값의
# 리스트를 딕셔너리로 만들기
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# 그리드 서치 클래스에 탐색 대상 모델과 params 변수를 전달해 그리드 서치 객체 만들기
# n_jobs: 병렬 실행에 사용할 CPU 코어 수. -1을 하면 시스템에 있는 모든 코어 사용
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

# GridSearchCV의 cv 기본값은 5, 각각 5-폴드 교차 검증 수행하므로 5 * 5개의 모델 훈련
gs.fit(train_input, train_target)

# best_estimator_: 여러 모델을 훈련하여 얻은 최적의 하이퍼파라미터로
#                  훈련셋 전체를 사용해 다시 훈련시킨 모델
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

# best_params_: 그리드 서치로 찾은 최적의 매개변수
print(gs.best_params_)

# cv_results_: 각 매개변수에서 수행한 교차 검증의 평균 점수
print(gs.cv_results_['mean_test_score'])

# 넘파이 argmax(): 가장 큰 값의 인덱스 추출
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [24]:
# 복잡한 매개변수 조합 탐색
# min_impurity_decrease(노드 분할 위한 불순도 감소 최소량), max_depth,
# min_samples_split(노드 나누기 위한 최소 샘플 수)
# np.arange(): 파이썬의 range와 같으나, 실수 가능
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
         }

# 만들어지는 모델의 수: 9 * 15 * 10 * 5(5-폴드) = 6750
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)

# 최상의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


### 랜덤 서치
위의 그리드 서치 시 탐색할 매개변수의 간격을 0.0001, 1 등으로 지정했는데 이렇게 간격을 둔 것에 특별한 근거가 없다. **랜덤 서치(Random Search)**는 매개변수 값의 목록을 전달하는 것이 아닌 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다. 싸이파이의 stats 서브 패키지에 있는 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는데, 이를 '균등 분포에서 샘플링한다'고 말한다. randint는 정숫값, uniform은 실숫값을 뽑는다

In [25]:
from scipy.stats import uniform, randint

# 0에서 10 사이의 범위를 갖는 randint 객체 만들고 10개의 숫자 샘플링
rgen = randint(0, 10)
rgen.rvs(10)

array([1, 8, 3, 1, 7, 3, 2, 3, 8, 0])

In [28]:
# 1000개 샘플링 후 각 숫자 개수 세기
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98,  89,  98,  93, 121, 109,  83, 115, 101,  93]))

In [34]:
# 탐색할 매개변수: min_samples_leaf(리프노드의 최소 샘플 수) 추가
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)
         }

# n_iter: 샘플링 횟수
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [35]:
# 최고의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

# 테스트 성능 확인
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8695428296438884
0.86
